In [1]:
"""%load_ext autoreload
%autoreload 2
%load_ext wurlitzer
"""
import json
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import xobjects as xo
import xpart as xp
import xtrack as xt
import xcoll as xc
import os
import yaml

In [2]:
context = xo.ContextCpu()
# context = xo.ContextCupy()
# context = xo.ContextPyopencl()

# On a modern CPU, we get ~5000 particle*turns/s
# So this script should take around half an hour
beam          =  2
plane         = 'V'

num_turns     = 20
num_particles = 20000
engine        = 'everest'

#path_in  = Path("/eos/project-c/collimation-team/machine_configurations/LHC_run3/2024")

# Load from json
line = xt.Line.from_json(os.path.expandvars('${HOME_TWOCRYST}/input_files/2024_Run3/flat_top_b2.json'))

Loading line from dict:   0%|          | 0/175742 [00:00<?, ?it/s]

Done loading line from dict.           


In [3]:
coll_file = os.path.expandvars('${HOME_TWOCRYST}/input_files/colldbs/flat_top.yaml')
with open(coll_file, 'r') as stream:
        coll_dict = yaml.safe_load(stream)['collimators']['b2']

In [4]:
end_s = line.get_length()

TCCS_name = 'tccs.5r3.b2'
TCCP_name = 'tccp.4l3.b2'
TARGET_name = 'target.4l3.b2'
PIXEL_name = 'pixel.detector'
TCP_name = 'tcp.d6r7.b2'
TCLA_name = 'tcla.a5l3.b2'

d_pix = 1 # [m]
ydim_PIXEL = 0.01408
xdim_PIXEL = 0.04246

TCCS_loc = end_s - 6773.7 #6775
TCCP_loc = end_s - 6653.3 #6655

dx = 1e-11
TARGET_loc = end_s - (6653.3 + coll_dict[TCCP_name]["length"]/2 + coll_dict[TARGET_name]["length"]/2 + dx)
PIXEL_loc = end_s - (6653.3 - coll_dict[TCCP_name]["length"]/2 - d_pix)
TCP_loc = line.get_s_position()[line.element_names.index(TCP_name)]
TCLA_loc = line.get_s_position()[line.element_names.index(TCLA_name)]


line.insert_element(at_s=TCCS_loc, element=xt.Marker(), name=TCCS_name)
line.insert_element(at_s=TCCS_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name=TCCS_name+'_aper')
line.insert_element(at_s=TCCP_loc, element=xt.Marker(), name=TCCP_name)
line.insert_element(at_s=TCCP_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name=TCCP_name+'_aper')
line.insert_element(at_s=TARGET_loc, element=xt.Marker(), name=TARGET_name)
line.insert_element(at_s=TARGET_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name= TARGET_name + '_aper')
line.insert_element(at_s=PIXEL_loc, element=xt.Marker(), name=PIXEL_name)

Slicing line:   0%|          | 0/175742 [00:00<?, ?it/s]

Slicing line:   0%|          | 0/175745 [00:00<?, ?it/s]

Slicing line:   0%|          | 0/175748 [00:00<?, ?it/s]

Slicing line:   0%|          | 0/175751 [00:00<?, ?it/s]

In [6]:
# Initialise collmanager
# coll_manager = xc.CollimatorManager.from_yaml(path_in / 'colldbs' / f'levelling.20.yaml', line=line,
#                                               beam=beam, ignore_crystals=False, record_impacts=False)

coll_manager = xc.CollimatorManager.from_yaml(coll_file, line=line,
                                              beam=beam, ignore_crystals=False, record_impacts=['tcp.c6r7.b2', 'tcp.d6r7.b2'])

# Install collimators into line
if engine == 'everest':
    coll_names = coll_manager.collimator_names
    coll_manager.install_everest_collimators(names =  [name for name in coll_names if name not in [TARGET_name]], verbose=True)
elif engine == 'black':
    coll_manager.install_black_absorbers(verbose=True)
else:
    raise ValueError(f"Unknown scattering engine {engine}!")

# Aperture model check
print('\nAperture model check after introducing collimators:')
df_with_coll = line.check_aperture()
assert not np.any(df_with_coll.has_aperture_problem)

Compiling ContextCpu kernels...


cd68becbd54a4ca7ab4e8f0c1c6ba32c.c: In function 'scatter_cry':
cd68becbd54a4ca7ab4e8f0c1c6ba32c.c:10560:28: warning: unused variable 'x_P' [-Wunused-variable]
10560 |                     double x_P = -s_P_tmp*sin(tilt_int) + x_P_tmp*cos(tilt_int);
      |                            ^~~
cd68becbd54a4ca7ab4e8f0c1c6ba32c.c:10559:28: warning: unused variable 's_P' [-Wunused-variable]
10559 |                     double s_P = s_P_tmp*cos(tilt_int) + x_P_tmp*sin(tilt_int);
      |                            ^~~
cd68becbd54a4ca7ab4e8f0c1c6ba32c.c:10457:18: warning: unused variable 'cry_spTilt' [-Wunused-variable]
10457 |     double const cry_spTilt = sin(cry_tilt);
      |                  ^~~~~~~~~~
cd68becbd54a4ca7ab4e8f0c1c6ba32c.c:10447:12: warning: unused variable 'offset' [-Wunused-variable]
10447 |     double offset   = everest->coll->offset;
      |            ^~~~~~


Done compiling ContextCpu kernels.
Installing tcpcv.a6r7.b2        as EverestCrystal


Slicing line:   0%|          | 0/175750 [00:00<?, ?it/s]

Installing tcpch.a5r7.b2        as EverestCrystal


...

Slicing line:   0%|          | 0/175356 [00:00<?, ?it/s]


Aperture model check after introducing collimators:


Checking aperture:   0%|          | 0/175359 [00:00<?, ?it/s]

Done checking aperture.           
0 thin elements miss associated aperture (upstream):
[]
13 thick elements miss associated aperture (upstream or downstream):
['drift_8443..0',
 'drift_8451..0',
 'drift_8648..0',
 'drift_8656..0',
 'drift_8658..1',
 'drift_8739..0',
 'drift_8741..1',
 'drift_8892..0',
 'drift_8894..1',
 'drift_8975..0',
 'drift_8977..1',
 'drift_28834..0',
 'drift_28836..1']


AssertionError: 

In [ ]:
coll_manager.build_tracker()

In [5]:
coll_manager.set_openings({'tcp.d6l7.b1': 5, 'tcp.c6l7.b1': 5, 'tcpcv.a6l7.b1': 5.5})

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [6]:
line['tcpch.a4l7.b1'].active = True
line['tcpcv.a6l7.b1'].active = True
line['tcp.b6l7.b1'].record_touches = True
line['tcp.c6l7.b1'].record_touches = True
line['tcp.d6l7.b1'].record_touches = True
line['tdisa.a4l2.b1'].record_touches = True

In [7]:
line['tcpch.a4l7.b1'].to_dict()

{'__class__': 'EverestCrystal',
 'length': 0.004,
 'active': 1,
 'record_touches': 0,
 'record_interactions': 0,
 'align_angle': 1.2112695362072954e-05,
 '_critical_angle': 0.0,
 'xdim': 0.002,
 'ydim': 0.05,
 'thick': 0.0,
 'miscut': 0.0,
 'rutherford_rng': {'__class__': 'RandomRutherford',
  'lower_val': 0.0009982,
  'upper_val': 0.02,
  'A': 0.0016160247264725453,
  'B': 166.49518410000002,
  'Newton_iterations': 7},
 '_tracking': 0,
 'angle': 0.0,
 'jaw': [0.0020206608911066404, 0.02499999999999991],
 'tilt': 0.0,
 'side': 'left',
 'lattice': 'strip',
 'material': {'__class__': 'CrystalMaterial',
  'Z': 14.0,
  'A': 28.08,
  'density': 2.33,
  'excitation_energy': 1.73e-07,
  'nuclear_radius': 0.441,
  'nuclear_elastic_slope': 120.14,
  'cross_section': array([6.64e-01, 4.30e-01, 0.00e+00, 0.00e+00, 0.00e+00, 3.90e-04]),
  'hcut': 0.02,
  'name': 'Silicon',
  '_only_mcs': 0,
  'crystal_radiation_length': 0.0937,
  'crystal_nuclear_length': 0.4652,
  'crystal_plane_distance': 9.6e-0

In [8]:
tw = line.twiss()

In [9]:
coll = 'tcpch.a4l7.b1'
ang = line[coll].angle
ref = np.cos(ang)* tw.rows[coll]['x'] + np.sin(ang)* tw.rows[coll]['y']
sigx = np.sqrt(3.5E-6*tw.rows[coll]['betx']/7247.36468857)
sigy = np.sqrt(3.5E-6*tw.rows[coll]['bety']/7247.36468857)
sig = np.sqrt(np.cos(ang)**2 * sigx**2 + np.sin(ang)**2 * sigy**2)
print(ref + sig*5)

[0.00202066]


In [10]:
part = xc.generate_pencil_on_collimator(line, 'tcp.d6l7.b1', 100, nemitt_x=3.5e-6, nemitt_y=3.5e-6)

Collimator tcp.d6l7.b1 is diverging.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [11]:
coll_manager._io_buffer

<BufferNumpy 999992/241000752>

In [12]:
part._xobject._io_buffer

AttributeError: 'ParticlesData' object has no attribute '_io_buffer'

In [13]:
coll_manager.impacts._index.buffer_id

0

In [14]:
# # Optimise the line
# line.optimize_for_tracking()
# idx = line.element_names.index(tcp)
# part.at_element = idx
# part.start_tracking_at_element = idx

# line.discard_tracker()
# line.build_tracker(_context=xo.ContextCpu(omp_num_threads=28))

# Track
coll_manager.enable_scattering()
line.track(part, num_turns=20, time=True, with_progress=1)
coll_manager.disable_scattering()
print(f"Done tracking in {line.time_last_track:.1f}s.")

# line.discard_tracker()
# line.build_tracker(_context=xo.ContextCpu())

Tracking:   0%|          | 0/20 [00:00<?, ?it/s]

Error: buffer_id mismatch!
Error: buffer_id mismatch!
Error: buffer_id mismatch!
Error: buffer_id mismatch!
Done tracking in 0.0s.


In [15]:
# Save lossmap to json, which can be loaded, combined (for more statistics),
# and plotted with the 'lossmaps' package
line_is_reversed = True if f'{beam}' == '2' else False
ThisLM = xc.LossMap(line, line_is_reversed=line_is_reversed, part=part)
path_out = Path.cwd()
ThisLM.save_summary(file=Path(path_out, f'coll_summary_B{beam}{plane}.out'))
# Save a summary of the collimator losses to a text file
print(ThisLM.summary)

         collname  nabs  length             s               type
0      tcl.4r1.b1   0.0   1.000    150.030000  EverestCollimator
1      tcl.5r1.b1   0.0   1.000    184.357000  EverestCollimator
2      tcl.6r1.b1   0.0   1.000    219.013000  EverestCollimator
3    tctph.4l2.b1   0.0   1.000   3213.903584  EverestCollimator
4    tctpv.4l2.b1   0.0   1.000   3215.903583  EverestCollimator
5   tdisa.a4l2.b1  89.0   1.565   3249.693583  EverestCollimator
6   tdisb.a4l2.b1   0.0   1.565   3251.273583  EverestCollimator
7   tdisc.a4l2.b1   0.0   1.565   3252.853583  EverestCollimator
8       tclia.4r2   0.0   1.000   3403.984583  EverestCollimator
9    tclib.6r2.b1   0.0   1.000   3560.090583  EverestCollimator
10  tcld.a11r2.b1   0.0   0.600   3758.149975  EverestCollimator
11     tcp.6l3.b1   0.0   0.600   6487.671299  EverestCollimator
12    tcsg.5l3.b1   0.0   1.000   6520.992797  EverestCollimator
13    tcsg.4r3.b1   0.0   1.000   6707.575797  EverestCollimator
14   tcsg.a5r3.b1   0.0  

In [13]:
coll_manager.impacts.to_pandas()

,turn,collimator,interaction_type,ds,parent_id,parent_x,parent_px,parent_y,parent_py,parent_zeta,...,child_y,child_py,child_zeta,child_delta,child_energy,child_mass,child_charge,child_z,child_a,child_pdgid


In [15]:
coll_manager.impacts.to_pandas().columns

Index(['turn', 'collimator', 'interaction_type', 'ds', 'parent_id', 'parent_x',
       'parent_px', 'parent_y', 'parent_py', 'parent_zeta', 'parent_delta',
       'parent_energy', 'parent_mass', 'parent_charge', 'parent_z', 'parent_a',
       'parent_pdgid', 'child_id', 'child_x', 'child_px', 'child_y',
       'child_py', 'child_zeta', 'child_delta', 'child_energy', 'child_mass',
       'child_charge', 'child_z', 'child_a', 'child_pdgid'],
      dtype='object')

In [14]:
coll_manager.lossmap(part, file=None)

{'collimator': {'s': [219.013,
   6520.992797083998,
   19789.18438252112,
   19791.184382375122,
   19793.184382229123,
   19832.67888061313,
   19891.906380613127,
   19895.906380613127,
   19917.23638061313,
   19989.16238061313,
   19991.16238061313,
   19995.16238061313,
   20086.41838061313,
   20102.41838061313,
   20108.41838061313,
   20148.089380613128,
   20178.96338010613,
   20212.23237811213,
   20231.86037811213,
   26510.938177061144,
   26512.938176726144],
  'name': ['tcl.6r1.b1',
   'tcsg.5l3.b1',
   'tcp.d6l7.b1',
   'tcp.c6l7.b1',
   'tcp.b6l7.b1',
   'tcsg.a6l7.b1',
   'tcsg.b5l7.b1',
   'tcsg.a5l7.b1',
   'tcsg.d4l7.b1',
   'tcspm.b4l7.b1',
   'tcsg.a4l7.b1',
   'tcsg.a4r7.b1',
   'tcsg.b5r7.b1',
   'tcsg.d5r7.b1',
   'tcspm.e5r7.b1',
   'tcla.a6r7.b1',
   'tcla.b6r7.b1',
   'tcla.c6r7.b1',
   'tcla.a7r7.b1',
   'tctph.4l1.b1',
   'tctpv.4l1.b1'],
  'length': [1.0,
   1.0,
   0.6,
   0.6,
   0.6,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   

In [12]:
pdb on

Automatic pdb calling has been turned ON


In [9]:
coll_manager.lossmap(part)

{'collimator': {'s': [19789.18438252112,
   19791.184382375122,
   19793.184382229123,
   19832.67888061313,
   19891.906380613127,
   19895.906380613127,
   19917.23638061313,
   19989.16238061313,
   19991.16238061313,
   19995.16238061313,
   20086.41838061313,
   20102.41838061313,
   20108.41838061313,
   20143.02338061313,
   20148.089380613128,
   20212.23237811213,
   26512.938176726144],
  'name': ['tcp.d6l7.b1',
   'tcp.c6l7.b1',
   'tcp.b6l7.b1',
   'tcsg.a6l7.b1',
   'tcsg.b5l7.b1',
   'tcsg.a5l7.b1',
   'tcsg.d4l7.b1',
   'tcspm.b4l7.b1',
   'tcsg.a4l7.b1',
   'tcsg.a4r7.b1',
   'tcsg.b5r7.b1',
   'tcsg.d5r7.b1',
   'tcspm.e5r7.b1',
   'tcspm.6r7.b1',
   'tcla.a6r7.b1',
   'tcla.c6r7.b1',
   'tctpv.4l1.b1'],
  'length': [0.6,
   0.6,
   0.6,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0],
  'n': [3873.0,
   4.0,
   72.0,
   208.0,
   246.0,
   144.0,
   46.0,
   11.0,
   51.0,
   37.0,
   21.0,
   14.0,
   4.